# Experiment 023: Stacking Meta-Learner

**Based on evaluator feedback:**
- exp_022 (multi-seed ensemble) CV 0.0901 is 45% worse than best - DO NOT SUBMIT
- CV-LB correlation is 0.994 - lower CV = lower LB
- Only 1 submission remaining - must beat CV 0.0623 to submit

**Approach:**
- Use exp_004 architecture (best CV 0.0623) as primary model
- NO modifications - just replicate exp_004 exactly
- If CV matches 0.0623, we have a valid submission candidate

**TEMPLATE COMPLIANCE**: Last 3 cells are EXACTLY as template, NO cells after them.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print("Setup complete")

Setup complete


In [2]:
# --- UTILITY FUNCTIONS ---
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature sets
SPANGE_DF = load_features('spange_descriptors')
ACS_PCA_DF = load_features('acs_pca_descriptors')
print(f"Spange: {SPANGE_DF.shape}, ACS_PCA: {ACS_PCA_DF.shape}")

Spange: (26, 13), ACS_PCA: (24, 5)


In [3]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [4]:
# --- DUAL FEATURIZER (Spange + ACS_PCA) - EXACT COPY FROM exp_004 ---
class DualFeaturizer:
    """Featurizer that builds features from both Spange and ACS_PCA descriptors."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange = SPANGE_DF
        self.acs_pca = ACS_PCA_DF

    def _build_process_features(self, X):
        """Build process features with Arrhenius kinetics."""
        rt = X['Residence Time'].values.astype(np.float64).reshape(-1, 1)
        temp = X['Temperature'].values.astype(np.float64).reshape(-1, 1)
        
        # Arrhenius kinetic features
        temp_k = temp + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(rt + 1e-6)
        interaction = inv_temp * log_time
        
        return np.hstack([rt, temp, inv_temp, log_time, interaction])

    def _get_solvent_features(self, X, feature_df, flip=False):
        """Get solvent features from a feature dataframe."""
        if self.mixed:
            A = feature_df.loc[X["SOLVENT A NAME"]].values
            B = feature_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)  # NOT divided by 100!
            if flip:
                return B * (1 - pct) + A * pct
            else:
                return A * (1 - pct) + B * pct
        else:
            return feature_df.loc[X["SOLVENT NAME"]].values

    def featurize_spange(self, X, flip=False):
        """Build features using Spange descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.spange, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)  # NOT divided by 100!
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

    def featurize_acs(self, X, flip=False):
        """Build features using ACS_PCA descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.acs_pca, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)  # NOT divided by 100!
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

In [5]:
# --- PER-TARGET HETEROGENEOUS MODEL - EXACT COPY FROM exp_004 ---
class PerTargetModel(BaseModel):
    """Per-target heterogeneous model with combined features.
    
    EXACT exp_004 architecture:
    - SM: HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
    - Products: ExtraTreesRegressor (n_estimators=500, min_samples_leaf=2, max_depth=10)
    - Feature weighting: 0.8 acs_pca + 0.2 spange
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = DualFeaturizer(mixed=(data=='full'))
        self.targets = ['Product 2', 'Product 3', 'SM']
        
        # Scalers for each feature set
        self.scaler_spange = StandardScaler()
        self.scaler_acs = StandardScaler()
        
        # Models: {target: {feature_set: model}}
        self.models = {}
        
        # Feature weights: 0.8 acs_pca + 0.2 spange
        self.acs_weight = 0.8
        self.spange_weight = 0.2

    def train_model(self, X_train, y_train):
        # Build features
        X_spange = self.featurizer.featurize_spange(X_train)
        X_acs = self.featurizer.featurize_acs(X_train)
        y = y_train.values
        
        if self.data_type == 'full':
            # Data augmentation with flipped features
            X_spange_flip = self.featurizer.featurize_spange(X_train, flip=True)
            X_acs_flip = self.featurizer.featurize_acs(X_train, flip=True)
            X_spange = np.vstack([X_spange, X_spange_flip])
            X_acs = np.vstack([X_acs, X_acs_flip])
            y = np.vstack([y, y])
        
        # Scale features
        X_spange_scaled = self.scaler_spange.fit_transform(X_spange)
        X_acs_scaled = self.scaler_acs.fit_transform(X_acs)
        
        # Train per-target models
        for i, target in enumerate(self.targets):
            self.models[target] = {}
            y_target = y[:, i]
            
            if target == 'SM':
                model_spange = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04,
                    random_state=42
                )
                model_acs = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04,
                    random_state=42
                )
            else:
                model_spange = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    random_state=42, n_jobs=-1
                )
                model_acs = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    random_state=42, n_jobs=-1
                )
            
            model_spange.fit(X_spange_scaled, y_target)
            model_acs.fit(X_acs_scaled, y_target)
            
            self.models[target]['spange'] = model_spange
            self.models[target]['acs'] = model_acs

    def predict(self, X):
        # Build features
        X_spange = self.featurizer.featurize_spange(X)
        X_acs = self.featurizer.featurize_acs(X)
        
        # Scale features
        X_spange_scaled = self.scaler_spange.transform(X_spange)
        X_acs_scaled = self.scaler_acs.transform(X_acs)
        
        # Predict per target
        predictions = []
        for target in self.targets:
            pred_spange = self.models[target]['spange'].predict(X_spange_scaled)
            pred_acs = self.models[target]['acs'].predict(X_acs_scaled)
            
            # Weighted combination: 0.8 acs + 0.2 spange
            pred = self.acs_weight * pred_acs + self.spange_weight * pred_spange
            predictions.append(pred)
        
        # Stack and clip
        predictions = np.column_stack(predictions)
        predictions = np.clip(predictions, 0, 1)
        
        return torch.tensor(predictions)

print("PerTargetModel defined (exact exp_004 copy)")

PerTargetModel defined (exact exp_004 copy)


In [6]:
# Quick validation test to verify we match exp_004
print("Testing PerTargetModel (exp_004 replica)...")
X_test, Y_test = load_data("single_solvent")

errors = []
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_test, Y_test)):
    if i >= 5: break
    model = PerTargetModel(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    solvent = test_X['SOLVENT NAME'].iloc[0]
    print(f"Single Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nexp_004 replica quick test MAE: {np.mean(errors):.4f}")
print("Expected: ~0.09 for first 5 folds (exp_004 had similar)")

Testing PerTargetModel (exp_004 replica)...


Single Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1446


Single Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.0962


Single Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0369


Single Fold 3 (Acetonitrile): MAE = 0.0813


Single Fold 4 (Acetonitrile.Acetic Acid): MAE = 0.1077

exp_004 replica quick test MAE: 0.0934
Expected: ~0.09 for first 5 folds (exp_004 had similar)


In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PerTargetModel(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.89s/it]

2it [00:03,  1.95s/it]

3it [00:05,  1.94s/it]

4it [00:07,  1.94s/it]

...

21it [00:41,  1.99s/it]

22it [00:43,  1.98s/it]

23it [00:45,  1.96s/it]

24it [00:47,  1.95s/it]

24it [00:47,  1.97s/it]

In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PerTargetModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:02,  2.49s/it]

2it [00:04,  2.49s/it]

3it [00:07,  2.52s/it]

4it [00:10,  2.56s/it]

...

10it [00:24,  2.44s/it]

11it [00:27,  2.43s/it]

12it [00:29,  2.44s/it]

13it [00:31,  2.42s/it]

13it [00:31,  2.46s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [10]:
# Calculate CV score - THIS CELL MUST BE REMOVED BEFORE SUBMISSION
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Single solvent CV
single_preds = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
single_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_single, Y_single)):
    single_true.append(test_Y.values)
single_true = np.vstack(single_true)
single_mae = np.mean(np.abs(single_preds - single_true))
print(f"Single solvent CV MAE: {single_mae:.4f}")

# Full data CV
full_preds = submission_full_data[['target_1', 'target_2', 'target_3']].values
full_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full)):
    full_true.append(test_Y.values)
full_true = np.vstack(full_true)
full_mae = np.mean(np.abs(full_preds - full_true))
print(f"Full data CV MAE: {full_mae:.4f}")

# Combined
combined_mae = (single_mae + full_mae) / 2
print(f"\nCombined CV MAE: {combined_mae:.4f}")
print(f"\nexp_004 original CV: 0.0623")
print(f"Difference: {combined_mae - 0.0623:.4f}")

Single solvent CV MAE: 0.0677
Full data CV MAE: 0.0943

Combined CV MAE: 0.0810

exp_004 original CV: 0.0623
Difference: 0.0187
